## Goals

- Read an ASDF file
- Use tools for looking at the file and searching for attributes
- Display an image contained in the file
- Access and modify metadata
- Write the file to disk

The file jwst.asdf is presumed to be in the tutorial's data/ directory. This is the combined resampled product of two JWST NIRCam observations.

In [ ]:
import numpy as np
import asdf

from matplotlib import pyplot as plt
%matplotlib widget

Load ASDF file
--------------

In [ ]:
af = asdf.open('../data/jwst.asdf')

The file can be rendered using the **info** method

In [ ]:
af.info()

In [ ]:
# The file has lots of content so some options are needed to see it all

af.info(max_rows=200)

**info** is also a utility function. If called like that it can render specific nodes in the tree.

And there's a command line utility **asdftool info** which allows looking at files outside a Python session.

In [ ]:
asdf.info(af['meta']['instrument'])

Another useful utility is **search** - search by key, value or type.

In [ ]:
# Search for attributes with exposure in their name
af.search('exposure')

##### Accessing attributes 

Attributes are accessed using a dict-like syntax.

In [ ]:
# Accessing attributes: assign an attribute value to a variable
elapsed_time = af.tree['meta']['exposure']['elapsed_exposure_time']
print(elapsed_time)

As a side note, both JWST and Roman datamodels, which are built as ASDF extensions, add utilities that allow access through dot notation, presuming the file was opened as a datmodel

In [ ]:
from stdatamodels.jwst import datamodels

dm = datamodels.open('../data/jwst.asdf')
dm.meta.exposure.elapsed_exposure_time

In [ ]:
# Now load image and display it
im = af.tree['data']
print(im.shape)

In [ ]:
# Obviously we need to adjust the stretch on this. I've already found a reasonable one.
fig, ax = plt.subplots()
ax.imshow(im, vmin=0, vmax=5, origin='lower')

In [ ]:
# To use a log transfer function means getting rid of all NaNs
nim = np.array(im) # Existing image is only read only and not a numpy array
nim[np.isnan(nim)] = 5
nim[nim<=0] = 1

fig1, ax1 = plt.subplots()
ax1.imshow(np.log(nim), vmin=0, vmax=2.5, origin='lower')


Modifying the contents
----------------------

In [ ]:
# Arbitrarily change the elapsed_exposure_time
af.tree['meta']['exposure']['elapsed_exposure_time'] = 1800
print(af.tree['meta']['exposure']['elapsed_exposure_time'])

In [ ]:
# Replace data attribute with the log version of the image.
af.tree['data'] = np.log(nim)

In [ ]:
# Create new meta attribute
af.tree['meta']['inspected_by'] = 'Jacques Clouseau'

Saving the modified contents
----------------------------

In [ ]:
# Save to a different file
af.write_to('jwst_modified.asdf')

In [ ]:
ls

Exercises
---------

1. Search for the name of the PI ("Klaus") and replace the attribute containing
   it with your own name. Type "af.search?" to show how to search values instead of attribute names.
3. Use the above log image and set all pixels in rows 3000:3010 and columns 4000:4010
   to a value of 2.5, and replace the data attribute with the modified log image.
5. Save the result to a new file named "jwst_exercise.asdf"
6. Load that new file and display the data image
7. Print out the value of the attribute that contains the name of the PI
   to confirm the edits were saved